In [25]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time


# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


In [26]:
# Define paramaters for the model
learning_rate = 0.001
batch_size = 128
n_epochs = 20

In [27]:
# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('./data/mnist', one_hot=True) 


Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [28]:
# Step 1b: Define parameters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 25

In [29]:
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# each lable is one hot vector.
X = tf.placeholder(tf.float32, [batch_size, 784], name='X_placeholder') 
Y = tf.placeholder(tf.int32, [batch_size, 10], name='Y_placeholder')

In [30]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name='weights')
b = tf.Variable(tf.zeros([1, 10]), name="bias")

In [31]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
logits = tf.matmul(X, w) + b 


In [32]:
# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(entropy) # computes the mean over all the examples in the batch

In [33]:
# Step 6: define training op
# using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


In [34]:
# Create a model saver
saver = tf.train.Saver()

# Create a log directory for tensorboard
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "./{}/run-{}/".format(root_logdir, now)

In [36]:


# Step 7: Training
with tf.Session() as sess:
    # to visualize using TensorBoard
    writer = tf.summary.FileWriter('./graphs/logistic_reg', sess.graph)

    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch, Y:Y_batch}) 
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        save_path = saver.save(sess, "./models/log_regression.ckpt")
        
    best_weights = w.eval()
    print('Total time: {0} seconds'.format(time.time() - start_time))
    print ('Saving File')
    save_path = saver.save(sess, "./models/log_regression.ckpt")
    print('Optimization Finished!') # should be around 0.35 after 25 epochs
    
    writer.close()

Average loss epoch 0: 0.368197188771
Average loss epoch 1: 0.295828258936
Average loss epoch 2: 0.284424233791
Average loss epoch 3: 0.281339878247
Average loss epoch 4: 0.27461632065
Average loss epoch 5: 0.275837533541
Average loss epoch 6: 0.270724972072
Average loss epoch 7: 0.268915197337
Average loss epoch 8: 0.265146222773
Average loss epoch 9: 0.263856327166
Average loss epoch 10: 0.263401987183
Average loss epoch 11: 0.262411679369
Average loss epoch 12: 0.262011539808
Average loss epoch 13: 0.260671258355
Average loss epoch 14: 0.261939991465
Average loss epoch 15: 0.26189477542
Average loss epoch 16: 0.25797843822
Average loss epoch 17: 0.256840896903
Average loss epoch 18: 0.256169855716
Average loss epoch 19: 0.255518775546
Average loss epoch 20: 0.255584608946
Average loss epoch 21: 0.253940271157
Average loss epoch 22: 0.256339970342
Average loss epoch 23: 0.256150481011
Average loss epoch 24: 0.250612888737
Total time: 15.8112399578 seconds
Saving File
Optimization Fini

In [37]:
best_weights.shape

(784, 10)

In [1]:
 # test the model

#     preds = tf.nn.softmax(logits)
#     correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
#     accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
    
#     n_batches = int(mnist.test.num_examples/batch_size)
#     total_correct_preds = 0
    
#     for i in range(n_batches):
#         X_batch, Y_batch = mnist.test.next_batch(batch_size)
#         accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch})
#         print accuracy_batch
#         print accuracy_batch.shape
        #total_correct_preds = total_correct_preds+ accuracy_batch

    #print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))